## AdventureWorks EDA Using Spark SQL aggregate functions from Python

![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)

More examples are available on the Spark website: http://spark.apache.org/examples.html

PySpark API documentation: http://spark.apache.org/docs/latest/api/python/

## Author: Bryan Cafferky Copyright 08/20/2021

### Warning!!!

#### To run this code, you need to have uploaded the files and created the database tables - see Lesson 9 - Creating the SQL Tables on Databricks.  Link in video description to that video.

## Using Spark SQL from Python

#### Dataframe naming prefix convention:
##### 1st character is s for Spark DF
##### 2nd character is p for Python
##### 3rd and 4th character is df for dataframe
##### 5th = _ separator
##### rest is a meaningful name

##### spdf_salessummary = a Spark Python dataframe containing sales summary information.

In [0]:
spark.sql('use awproject')
spdf_salesinfo = spark.sql('select * from factinternetsales').dropna()

In [0]:
display(spdf_salesinfo.head(5))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,RevisionNumber,OrderQuantity,UnitPrice,ExtendedAmount,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber,OrderDate,DueDate,ShipDate
310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,1,1,3578.27,3578.27,0,0,2171.2942,2171.2942,3578.27,286.2616,89.4568,NULL,NULL,2010-12-29 00:00:00.000,2011-01-10 00:00:00.000,2011-01-05 00:00:00.000
346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,1,1,3399.99,3399.99,0,0,1912.1544,1912.1544,3399.99,271.9992,84.9998,NULL,NULL,2010-12-29 00:00:00.000,2011-01-10 00:00:00.000,2011-01-05 00:00:00.000
346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,1,1,3399.99,3399.99,0,0,1912.1544,1912.1544,3399.99,271.9992,84.9998,NULL,NULL,2010-12-29 00:00:00.000,2011-01-10 00:00:00.000,2011-01-05 00:00:00.000
336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,1,1,699.0982,699.0982,0,0,413.1463,413.1463,699.0982,55.9279,17.4775,NULL,NULL,2010-12-29 00:00:00.000,2011-01-10 00:00:00.000,2011-01-05 00:00:00.000
346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,1,1,3399.99,3399.99,0,0,1912.1544,1912.1544,3399.99,271.9992,84.9998,NULL,NULL,2010-12-29 00:00:00.000,2011-01-10 00:00:00.000,2011-01-05 00:00:00.000


#### Using SQL Aggregate Functions via agg()
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### More on using SQL Aggregate Functions
https://sparkbyexamples.com/spark/spark-sql-aggregate-functions/

### Just using SQL directly...

In [0]:
%sql

SELECT avg(SalesAmount) as avg_sales, 
       stddev(SalesAmount) as std_sales, 
       min(SalesAmount) as min_sales, 
       max(SalesAmount) as max_sales, 
       approx_count_distinct(ProductKey) as count_productkey, 
       approx_count_distinct(CustomerKey) as count_customerkey
FROM factinternetsales

avg_sales,std_sales,min_sales,max_sales,count_productkey,count_customerkey
486.0869105053777,928.489891980807,2.29,3578.27,161,17004


In [0]:
spdf_salesagg = spark.sql('''
SELECT avg(SalesAmount) as avg_sales, 
       stddev(SalesAmount) as std_sales, 
       min(SalesAmount) as min_sales, 
       max(SalesAmount) as max_sales, 
       approx_count_distinct(ProductKey) as count_productkey, 
       approx_count_distinct(CustomerKey) as count_customerkey
FROM factinternetsales''')

display(spdf_salesagg)

avg_sales,std_sales,min_sales,max_sales,count_productkey,count_customerkey
486.0869105053777,928.489891980807,2.29,3578.27,161,17004


### Look mom, just Python...

In [0]:
# import some more functions
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import avg
from pyspark.sql.functions import max, min
from pyspark.sql.functions import stddev

# Perform aggregations on the DataFrame
spdf_agg = spdf_salesinfo.agg(
    avg(spdf_salesinfo.SalesAmount).alias("avg_sales"), 
    stddev("SalesAmount").alias("stddev_sales"),
    min(spdf_salesinfo.SalesAmount).alias("min_sales"),
    max(spdf_salesinfo.SalesAmount).alias("max_sales"),
    countDistinct(spdf_salesinfo.ProductKey).alias("distinct_products"), 
    countDistinct(spdf_salesinfo['CustomerKey']).alias('distinct_customers')
)

# Convert the results to Pandas DataFrame
spdf_agg.toPandas()

Out[10]:

,avg_sales,stddev_sales,min_sales,max_sales,distinct_products,distinct_customers
0,486.086911,928.489892,2.29,3578.27,158,18484


In [0]:
type(spdf_agg)

Out[11]: pyspark.sql.dataframe.DataFrame

In [0]:
import pyspark.sql.functions

spdf_salesinfo.groupBy("ProductKey").avg("SalesAmount").toPandas()

Out[12]:

,ProductKey,avg(SalesAmount)
0,463,24.49
1,471,63.50
2,540,32.60
3,580,1700.99
4,588,769.49
...,...,...
153,535,24.99
154,562,2384.07
155,475,69.99
156,483,120.00


In [0]:
#  See https://sparkbyexamples.com/spark/using-groupby-on-dataframe/

import pyspark.sql.functions

# Notice, groupBy() before the agg() functions.
spdf_salesinfo.groupBy("PromotionKey").agg(
    avg("SalesAmount").alias("avg_sales"), 
    stddev("SalesAmount").alias("stddev_sales"),
    min(spdf_salesinfo.SalesAmount).alias("min_sales"),
    max(spdf_salesinfo.SalesAmount).alias("max_sales"),
    countDistinct(spdf_salesinfo.ProductKey).alias("distinct_products"), 
    countDistinct(spdf_salesinfo['CustomerKey']).alias('distinct_customers')
).toPandas()


Out[13]:

,PromotionKey,avg_sales,stddev_sales,min_sales,max_sales,distinct_products,distinct_customers
0,1,468.824267,919.211592,2.29,3578.27,156,18405
1,13,742.350000,0.000000,742.35,742.35,9,20
2,14,2384.070000,0.000000,2384.07,2384.07,4,13
3,2,946.756482,1050.885142,2.29,2443.35,19,1990


In [0]:
# Modifed example from https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.GroupedData.agg.html

from pyspark.sql import functions as F

lpls_salesagg = spdf_salesinfo.agg(F.min(spdf_salesinfo.SalesAmount), F.max(spdf_salesinfo.SalesAmount)).collect()
lpls_salesagg

Out[14]: [Row(min(SalesAmount)=2.29, max(SalesAmount)=3578.27)]

In [0]:
type(lpls_salesagg)

Out[15]: list

In [0]:
from pyspark.sql import functions as F

lpdf_salesagg = spdf_salesinfo.agg(F.min(spdf_salesinfo.SalesAmount), F.max(spdf_salesinfo.SalesAmount)).toPandas()
lpdf_salesagg

Out[16]:

,min(SalesAmount),max(SalesAmount)
0,2.29,3578.27


In [0]:
from pyspark.sql.functions import col

spdf_salesinfo.groupBy("SalesTerritoryKey").agg(
                   avg("SalesAmount"), 
                   stddev("SalesAmount"), 
                   min("SalesAmount"), 
                   max("SalesAmount"), 
                   countDistinct("ProductKey"), countDistinct("CustomerKey"),
                   countDistinct("ProductKey", "CustomerKey")).where(col("count(ProductKey)") > 1).toPandas()

Out[17]:

,SalesTerritoryKey,avg(SalesAmount),stddev_samp(SalesAmount),min(SalesAmount),max(SalesAmount),count(ProductKey),count(CustomerKey),"count(ProductKey, CustomerKey)"
0,1,405.856394,860.822489,2.29,3578.2700,157,3341,8969
1,6,259.559693,722.628771,2.29,3578.2700,151,1571,6653
2,3,150.041480,467.091251,2.29,2071.4196,15,8,20
3,5,313.816656,700.421726,2.29,2384.0700,25,12,39
4,9,678.980936,1074.124424,2.29,3578.2700,158,3591,13208
5,4,466.216944,917.382067,2.29,3578.2700,158,4450,12250
6,8,514.544416,918.170934,2.29,3578.2700,158,1780,5584
7,7,475.713874,887.339536,2.29,3578.2700,155,1810,5461
8,10,491.125429,900.427387,2.29,3578.2700,155,1913,6840
9,2,241.943267,644.381638,2.29,2294.9900,17,8,27
